# Educational Notebook for Homogeneous Coordinates in Robotics
This Notebook provides a framework to visualize a robotic joints with the underlying math for Transformations in homogeneous coordinates.\
This Notebook is written by Erik Wichmann

# Libaries Installation
Symbolic math with LaTeX output of Matrices via Sympy\


In [18]:
import sys
!{sys.executable} -m pip install sympy  

# Import all necessary Libraries

In [19]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from sympy import *


In [20]:
#3D Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.patches import FancyArrowPatch


## Extend Matplotlib to plot arrows in 3D space
Implementation of the class is based on the gitHub gist from WetHat at https://gist.github.com/WetHat/1d6cd0f7309535311a539b42cccca89c \
but modified to handle homogeneous coordinates from sympy.

In [21]:
class Arrow3D(FancyArrowPatch):

    def __init__(self, p0, p1, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz0 = map(float,p0.evalf()[:-1])
        self._xyz1 = map(float,p1.evalf()[:-1])       

    def draw(self, renderer):
        x0, y0, z0 = self._xyz0
        x1, y1, z1 = self._xyz1

        xs, ys, zs = proj_transform((x0, x1), (y0, y1), (z0, z1), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x0, y0, z0 = self._xyz0
        x1, y1, z1 = self._xyz1

        xs, ys, zs = proj_transform((x0, x1), (y0, y1), (z0, z1), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 


def _arrow3D(ax, p0, p1, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''
        
    arrow = Arrow3D(p0, p1, *args, **kwargs)
    ax.add_artist(arrow)


setattr(Axes3D, 'arrow3D', _arrow3D)

# Constructing a SCARA Robot
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/2d/KUKA_Industrial_Robot_KR10_SCARA.jpg/220px-KUKA_Industrial_Robot_KR10_SCARA.jpg" />

### Setup SCARA Transformation Matrices
All parameters in the matrices are implemented as math symbols and the actual value of $h_1$, $h_2$, etc. is defined in *h1_val*. The actual values of $h_1$, etc. get substituted just before drawing the actual robot, so you can change the geometry later by running for example *h1_val = 2* to reduce height $h_1$ to 2.
This is implemented this way for educational pourposes, so you can *pprint()* any transformation matrix of interest and read out the algebraic equation instead of the actual computed numbers.



In [23]:
#Predefine SCARA robot
#
q1, q2, q3, q4 = symbols('q1, q2, q3, q4')
l1, l2, l3 = symbols('l1, l2, l3')
h1, h2, h3 = symbols('h1, h2, h3')

q5, q6 = symbols('q5, q6')

h1_val = 5
h2_val = 0.5
h3_val = 2
l1_val = 4
l2_val = 2
l3_val = 4

T0_1 = Matrix([[cos(q1), -sin(q1), 0,  0],
              [ sin(q1),  cos(q1), 0,  0],
              [       0,        0, 1, h1],
              [       0,        0, 0,  1]])

T1_2 = Matrix([[ cos(q2), -sin(q2),  0, l1],
              [ -sin(q2), -cos(q2),  0,  0],
              [        0,        0, -1, h2],
              [        0,        0,  0,  1]])

T2_3 = Matrix([[1, 0, 0,     l2],
              [ 0, 1, 0,      0],
              [ 0, 0, 1, -h3+q3],
              [ 0, 0, 0,      1]])

T3_4 = Matrix([[cos(q4), -sin(q4), 0,  0],
              [ sin(q4),  cos(q4), 0,  0],
              [       0,        0, 1, l3],
              [       0,        0, 0,  1]])


# Transformation matrices for all joints
T0_2 = T0_1*T1_2
T0_3 = T0_1*T1_2*T2_3
T0_4 = T0_1*T1_2*T2_3*T3_4


simplify(T0_4)

Matrix([
[ cos(-q1 + q2 + q4), -sin(-q1 + q2 + q4),  0, l1*cos(q1) + l2*cos(q1 - q2)],
[-sin(-q1 + q2 + q4), -cos(-q1 + q2 + q4),  0, l1*sin(q1) + l2*sin(q1 - q2)],
[                  0,                   0, -1,       h1 + h2 + h3 - l3 - q3],
[                  0,                   0,  0,                            1]])

# Create 3D-Plotter via Matplotlib
This codeblock defines all necessary functions to plot the new coordinate system into the Matplotlib figure.



In [24]:
#fig, ax = plt.subplots(figsize=(5,5))
#ax = fig.add_subplot(111, projection='3d')

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
fig.tight_layout()


def plotRobotPart3D(T, p, showBase=True,*args, **kwargs):
    """
    T is the Transformation Matrix containing the Basis of the local coordinate system 
    p is the vector conecting the local basis to the next one 
    """
    
    if det(T).evalf()!=1:
        print("Determinant is not 1 so something went wrong!")
          
    origin = T*Matrix([0, 0, 0, 1])
    
    #plot xyz unit vectors   
    if showBase:
        ex = T*Matrix([1,0,0,1])
        ey = T*Matrix([0,1,0,1])
        ez = T*Matrix([0,0,1,1])
        ax.arrow3D(origin, ex, mutation_scale=5, arrowstyle="->", color='red')
        ax.arrow3D(origin, ey, mutation_scale=5, arrowstyle="->", color='green')
        ax.arrow3D(origin, ez, mutation_scale=5, arrowstyle="->", color='blue')
        
    p_w = T*p
    ax.arrow3D(origin, p_w, *args, **kwargs)

    
    
def interactiveRobot(q1_rotation1, q2_rotation2, q3_toolLift, q4_toolRotation, q5_customLift, q6_customRotation):
       
    #clear old drawing to refresh frame
    ax.clear()
    ax.set_xlim(-1,9)
    ax.set_ylim(-5,5)
    ax.set_zlim(0,10)
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    #convert degree into radian
    q1_rotation1 = 2*pi*q1_rotation1/360
    q2_rotation2 = 2*pi*q2_rotation2/360
    q4_toolRotation = 2*pi*q4_toolRotation/360
    q6_customRotation = 2*pi*q6_customRotation/360
        
    #zeroVector points to (0,0,0) in homogeneous coordinates
    zeroVector = Matrix([0,0,0,1])
    
    #Identity Matrix represents the Basis of the world coordinates
    worldBasis = Matrix([[1, 0, 0, 0],
                          [0, 1, 0, 0],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]])
    

    #Substitute all Symbols with the actual values 
    _T0_1 = T0_1.subs({q1:q1_rotation1, q2:q2_rotation2, q3:q3_toolLift, q4:q4_toolRotation,
                       q5:q5_customLift, q6:q6_customRotation,
                       h1:h1_val, h2:h2_val, h3:h3_val,
                       l1:l1_val, l2:l2_val, l3:l3_val})
    _T1_2 = T1_2.subs({q1:q1_rotation1, q2:q2_rotation2, q3:q3_toolLift, q4:q4_toolRotation,
                       q5:q5_customLift, q6:q6_customRotation,
                       h1:h1_val, h2:h2_val, h3:h3_val,
                       l1:l1_val, l2:l2_val, l3:l3_val})
    _T2_3 = T2_3.subs({q1:q1_rotation1, q2:q2_rotation2, q3:q3_toolLift, q4:q4_toolRotation,
                       q5:q5_customLift, q6:q6_customRotation,
                       h1:h1_val, h2:h2_val, h3:h3_val,
                       l1:l1_val, l2:l2_val, l3:l3_val})
    _T3_4 = T3_4.subs({q1:q1_rotation1, q2:q2_rotation2, q3:q3_toolLift, q4:q4_toolRotation,
                       q5:q5_customLift, q6:q6_customRotation,
                       h1:h1_val, h2:h2_val, h3:h3_val,
                       l1:l1_val, l2:l2_val, l3:l3_val})
    
    #calculate all world to joint n transformation Matrices
    _T0_2 = _T0_1 * _T1_2
    _T0_3 = _T0_1 * _T1_2 * _T2_3
    _T0_4 = _T0_1 * _T1_2 * _T2_3 * _T3_4
    
    #draw the Robot      
    
    plotRobotPart3D(worldBasis, _T0_1*zeroVector, True, color='brown',mutation_scale=15)
    plotRobotPart3D(_T0_1, _T1_2*zeroVector, True, color='brown',mutation_scale=15)
    plotRobotPart3D(_T0_2, _T2_3*zeroVector, True, color='brown',mutation_scale=10)
    plotRobotPart3D(_T0_3, _T3_4*zeroVector, True, color='black',mutation_scale=10)
    plotRobotPart3D(_T0_4, Matrix([.3, .3, .3, 1]), True, color='brown',mutation_scale=1)
 
    plt.show()
    pprint(_T0_4.evalf(2))
                           
        
_ = widgets.interact(interactiveRobot,
                     q1_rotation1=(-90,90,5),
                     q2_rotation2=(-135,135,5),
                     q3_toolLift=(-1,2,.1),
                     q4_toolRotation=(-360,360,5),
                     q5_customLift=(-5, 5, .2),
                     q6_customRotation=(-180, 180, 5))
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='q1_rotation1', max=90, min=-90, step=5), IntSlider(value…

# Plotting the Robot interactivly in 3D Space
By utillizing the interactive widgets in jupyter lab you can change the parameters of the robot live.
Currently there is a small bug that the 3D Plot clears when rotating it or starting it. 
*The Robot will be shown after moving one of the sliders.* \
It seems to be an issue only in 3D plots but matplotlib is the only 3D Plotter that doesn't require OpenGL or comparable Rendering engines, which can be hard to install in some enviroments like the RWTH jupyterhub and would decrease the educational value this notebook provides.

# Now it's your turn:
## Manipulate the Transformation Matrix
You can define now your own transformation matrix by setting *T0_1*, *T1_2*, *T2_3*, *T3_4* to whatever transformation matrix you want. Try changing the rotation axis of a joint or use $q_5$ and $q_6$ to add another functionality. Both values are currently unused!

use *pprint()* if you want to see what the your resulting matrix
use *simplify()* if you want to simplify complex trigonometric expressions of a matrix


In [25]:
# for Exaple do something like this if you want the second joint to rotate around the x-axis instead of z-axis

# 1st make the linking arm extendable with slider q5
T1_2 = Matrix([[1, 0, 0, l1+q5],
               [0, 1, 0,     0],
               [0, 0, 1,    h2],
               [0, 0, 0,     1]])

# 2nd rotate around x-axis by multiplying T1_2 with a rotational matrix
T1_2 = T1_2 * Matrix([[1,       0,        0,  0],
                      [ 0, cos(q2), -sin(q2), 0],
                      [ 0, sin(q2),  cos(q2), 0],
                      [ 0,       0,        0, 1]])
# 3rd flip basis by 180° around x
T1_2 = T1_2 * Matrix([[1,  0,  0, 0],
                      [0, -1,  0, 0],
                      [0,  0, -1, 0],
                      [0,  0,  0, 1]])

# state the variable name as last line of code and you get a nice LaTeX version of your matrix
T1_2


Matrix([
[1,        0,        0, l1 + q5],
[0, -cos(q2),  sin(q2),       0],
[0, -sin(q2), -cos(q2),      h2],
[0,        0,        0,       1]])